In [27]:
from ipywidgets import interact, fixed
import matplotlib.pyplot as plt
from utl.Validating import UnetValidatingApp
import SimpleITK as sitk
import numpy as np
import astra
import torch
import torchvision.transforms as transforms
import scipy as sp
import scipy.ndimage
import torch
from util.imageprocessing import rescale
import numpy as np



In [28]:

def display_images_with_alpha_numpy(image_z, alpha, fixed, moving):
    img = (1.0 - alpha)*fixed[image_z,:,:] + alpha*moving[image_z,:,:] 
    plt.figure(figsize=(16,9))
    plt.imshow(img,cmap=plt.cm.Greys_r);
    plt.axis('off')
    plt.show()

In [29]:
ValApp=UnetValidatingApp()

ValApp.getCase(222)

Testing on cuda.
initialize network with normal
initialize network with normal
the file name are 


 Small FOV 

('Data\\SFOV\\P00_CT0_SFOV_S074_0.nii', 'Data\\SFOV\\P00_CT0_SFOV_S074_1.nii', 'Data\\SFOV\\P00_CT0_SFOV_S074_2.nii')

 CTD 

('Data\\CTD\\P00_CTD_0.nii', 'Data\\CTD\\P00_CTD_1.nii', 'Data\\CTD\\P00_CTD_2.nii')

 Large FOV 

('Data\\LFOV\\P00_CT0_LFOV_S074_0.nii', 'Data\\LFOV\\P00_CT0_LFOV_S074_1.nii', 'Data\\LFOV\\P00_CT0_LFOV_S074_2.nii')


In [30]:
ValApp.projo.shape

(3, 512, 512)

In [31]:
import numpy as np
Orig=np.repeat(ValApp.projo,2,axis=0)
Orig.shape

(6, 512, 512)

In [32]:
interact(display_images_with_alpha_numpy, image_z=(0,ValApp.proji.shape[0] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(ValApp.proji), moving=fixed(Orig));

interactive(children=(IntSlider(value=2, description='image_z', max=5), FloatSlider(value=0.5, description='al…

In [33]:
a='ResultCNN_3/test_G.pth'
# b is the best , e,f next best
NNFileName=a
ValApp.doValidation(NNFileName)
ValApp.out.shape

(3, 512, 512)

In [34]:
interact(display_images_with_alpha_numpy, image_z=(0,ValApp.out.shape[0] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(ValApp.out), moving=fixed(ValApp.projo),  vmin=0.4, vmax=+1.1);

interactive(children=(IntSlider(value=1, description='image_z', max=2), FloatSlider(value=0.5, description='al…

In [35]:

def display_images_with_alpha_numpy(image_z, alpha, fixed, moving, vmin=[], vmax=[]):
    img = (1.0 - alpha)*fixed[image_z,:,:] + alpha*moving[image_z,:,:] 
    plt.figure(figsize=(16,9))
    plt.imshow(img,cmap=plt.cm.Greys_r, vmin=vmin,vmax=vmax);
    plt.axis('off')
    plt.show()

def display_images_with_alpha_numpy1(image_z, alpha, fixed, moving):
    img = (1.0 - alpha)*fixed[image_z,:,:] + alpha*moving[image_z,:,:] 
    plt.figure(figsize=(16,9))
    plt.imshow(img,cmap=plt.cm.Greys_r, vmin=-0.06,vmax=0.22);
    plt.axis('off')
    plt.show()
def display_images_with_alpha_numpy2(image_z, alpha, fixed, moving):
    img = (1.0 - alpha)*fixed[image_z,:,:] + alpha*moving[image_z,:,:] 
    plt.figure(figsize=(16,9))
    plt.imshow(img,cmap=plt.cm.Greys_r, vmin=-1,vmax=1);
    plt.axis('off')
    plt.show()

In [36]:
def elastixreg(f,m,img=None):
    fixed_image =  sitk.GetImageFromArray(f)
    moving_image = sitk.GetImageFromArray(m)
    

    elastixImageFilter = sitk.ElastixImageFilter()
    elastixImageFilter.SetFixedImage(fixed_image)
    elastixImageFilter.SetMovingImage(moving_image)
    elastixImageFilter.SetParameterMap(sitk.GetDefaultParameterMap("rigid"))
    elastixImageFilter.SetParameter('DefaultPixelValue', str(0))
    elastixImageFilter.SetParameter('FinalBSplineInterpolationOrder ', '3')
    elastixImageFilter.SetParameter('FixedImagePyramid ', 'FixedRecursiveImagePyramid')
    elastixImageFilter.SetParameter('MovingImagePyramid ', 'MovingRecursiveImagePyramid')
    elastixImageFilter.SetParameter('AutomaticTransformInitialization', 'true')
    elastixImageFilter.SetParameter('Interpolator', 'BSplineInterpolator')
    elastixImageFilter.SetParameter('NumberOfResolutions', '6')


    elastixImageFilter.Execute()

    if img is None: 
        RR=elastixImageFilter.GetResultImage()
        RR=sitk.GetArrayFromImage(RR)

    else:
        thirdImage=sitk.GetImageFromArray(img)
        transformParameterMap = elastixImageFilter.GetTransformParameterMap()

        transformix = sitk.TransformixImageFilter()
        transformix.SetTransformParameterMap(transformParameterMap)
        transformix.SetMovingImage(thirdImage)

        transformix.Execute()
        RR=transformix.GetResultImage()
        RR=sitk.GetArrayFromImage(RR)
    return RR

In [37]:
from utl.pytorch_morph import *
from skimage import exposure
from skimage.transform import resize
out=ValApp.out
V1f=np.zeros_like(out)

for Slc in range(3):



    # CTDref=CTDplan[Slc,:,:][np.newaxis,:].squeeze(0)
    # CTDref = CTDref.squeeze(0) - CTDref.squeeze(0).min()
    # CTDref=CTDref/CTDref.max()
    noisy=ValApp.out[Slc,:,:][np.newaxis,:].squeeze(0)
    # noisy =noisy.squeeze(0) - noisy.squeeze(0).min()
    # noisy=noisy/noisy.max()

    Oorig=ValApp.projo[Slc,:,:][np.newaxis,:].squeeze(0)

    # matched = match_histograms(noisy, CTDref)
    # d=unsharp_mask(matched, radius=4, amount=3)

    # a_blur=denoise_tv_chambolle(noisy, weight=1, multichannel=False)
    # c=noisy+(noisy-a_blur)*2
    
    
    # c=unsharp_mask(noisy, radius=4, amount=3)   
    # CTDref=unsharp_mask(CTDref, radius=3, amount=2)

    e = exposure.equalize_adapthist(noisy)
    # CTDrefforreg = exposure.equalize_adapthist(CTDref)
    
    
    
    # p5, p98 = np.percentile(noisy, (5, 98))
    # f = exposure.rescale_intensity(noisy, in_range=(p5, p98))
    
    
    
    
    CTSforreg=e
    
    RS=round(512*7/15)
    HRS=round(RS/2)
    x, y =np.meshgrid(np.linspace(-HRS,HRS,num=2*HRS),np.linspace(-HRS,HRS,num=2*HRS))
    dst = np.sqrt(x*x+y*y)
    
    # Initializing sigma and muu
    sigma = HRS/2-round(HRS*5/100)
    muu = 0.000
    
    # Calculating Gaussian array
    gauss = np.exp(-( (dst-muu)**2 / ( 2.0 * sigma**2 ) ) )
    Temp=resize(ValApp.proji[Slc,:,:],(2*HRS,2*HRS))
    # Out_Imprinted=c
    Out_Imprinted=noisy.copy()
    Out_Imprinted[256-HRS:256+HRS,256-HRS:256+HRS]=Temp * (gauss) + Out_Imprinted[256-HRS:256+HRS,256-HRS:256+HRS] * (1- gauss)
    V1=Out_Imprinted[np.newaxis,:]
    V1f[Slc,:,:]=V1
    # # V1 =V1.squeeze(0) - V1.squeeze(0).min()
    # # V1=V1/V1.max()
    # # V1=V1[np.newaxis,:]
    # V2=ValApp.out[Slc,:,:][np.newaxis,:]


    # RR = elastixreg(CTDrefforreg,CTSforreg,V1.squeeze(0))
    # RR = sitkreg(CTDref,V1.squeeze(0))

    # sitk.Show(sitk.GetImageFromArray(Images1))
    # RR=RR[np.newaxis,:]

    # # Structural element Sphere 3x3
    # strel = np.ones((30, 30))
    # origin = tuple(((np.array((strel.shape))-1)/2).astype(int))

    # # Pytorch tophat
    # image_tensor = torch.tensor(RR, dtype=torch.float)
    # strel_tensor = torch.tensor(strel, dtype=torch.bool)
    # tophat_image_pytorch = tophat_pytorch(image_tensor, strel_tensor, origin=origin)
    # tophat_image_pytorch=tophat_image_pytorch.numpy()
    # tophatrec_image_pytorch = tophat_recon_pytorch(image_tensor, strel_tensor, origin=origin,k=5)
    # tophatrec_image_pytorch=tophatrec_image_pytorch.numpy()

    # # Pytorch tophatrec
    # image_tensor = torch.tensor(CTDref[np.newaxis,:], dtype=torch.float)
    # strel_tensor = torch.tensor(strel, dtype=torch.bool)
    # tophat_image_pytorch_ref = tophat_pytorch(image_tensor, strel_tensor, origin=origin)
    # tophat_image_pytorch_ref=tophat_image_pytorch_ref.numpy()
    # tophatrec_image_pytorch_ref = tophat_recon_pytorch(image_tensor, strel_tensor, origin=origin,k=5)
    # tophatrec_image_pytorch_ref=tophatrec_image_pytorch_ref.numpy()

    # # V2=np.concatenate((CTDref[np.newaxis,:],RR,tophat_image_pytorch,tophat_image_pytorch_ref),axis=0)
    # V2=np.concatenate((CTDref[np.newaxis,:],RR),axis=0)
    # V2Tophat=np.concatenate((tophat_image_pytorch,tophat_image_pytorch_ref),axis=0)
    

    # # CTDplan_tophatrec[np.where(CTDplan_tophatrec<0)]=0
    # # Out_registered_tophatrec[np.where(Out_registered_tophatrec<0)]=0

    # CTDplan_tophatrec[Slc,:,:] = tophatrec_image_pytorch_ref
    # Out_registered[Slc,:,:]=V2[1,:,:]
    # Out_registered_tophat[Slc,:,:]=V2Tophat[0,:,:]
    
    # Out_registered_tophatrec[Slc,:,:]=tophatrec_image_pytorch
    
    # CTDplan_tophat[Slc,:,:]=V2Tophat[1,:,:]


    # print(Slc)
    # # sitk.Show(sitk.GetImageFromArray(V2))
    # # sitk.Show(sitk.GetImageFromArray(V2Tophat))

interact(display_images_with_alpha_numpy, image_z=(0,V1f.shape[0] - 1), alpha=(0.0,1.0,0.05), fixed = fixed(V1f), moving=fixed(out), vmin=0.4, vmax=+1.1);


interactive(children=(IntSlider(value=1, description='image_z', max=2), FloatSlider(value=0.5, description='al…

In [42]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error as MSEE
from skimage.metrics import peak_signal_noise_ratio as PSNR

print(MSEE(ValApp.projo[1,:],ValApp.out[1,:]))
print(PSNR(ValApp.projo[1,:],ValApp.out[1,:]))
print(ssim(ValApp.projo[1,:],ValApp.out[1,:]))

0.0005715642302251058
32.42934958155982
0.9727949590752791


In [39]:
ValApp.projo.shape

(3, 512, 512)